<a href="https://colab.research.google.com/github/amitis94/080258/blob/main/efficientdet/tf2/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientDet Tutorial: inference, eval, and training



<table align="left"><td>
  <a target="_blank"  href="https://github.com/google/automl/blob/master/efficientdet/tf2/tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/google/automl/blob/master/efficientdet/tf2/tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

# 0. Install and view graph.

## 0.1 Install package and download source code/image.



In [2]:
!python --version

Python 3.10.12


In [3]:
%%capture
#@title
import os
import sys
import tensorflow as tf

# Download source code.
if "efficientdet" not in os.getcwd():
  !git clone --depth 1 https://github.com/google/automl
  os.chdir('automl/efficientdet')
  sys.path.append('.')
  !pip install -r requirements.txt
  !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
else:
  !git pull

In [4]:
MODEL = 'efficientdet-lite0'  #@param

def download(m):
  if m not in os.listdir():
    if m.find('lite'):
      !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tgz
      !tar zxf {m}.tgz
    else:
      !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz
      !tar zxf {m}.tar.gz
  ckpt_path = os.path.join(os.getcwd(), m)
  return ckpt_path

# Download checkpoint.
ckpt_path = download(MODEL)
print('Use model in {}'.format(ckpt_path))

# Prepare image and visualization settings.
image_url =  'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'#@param
image_name = 'img.png' #@param
!wget {image_url} -O img.png
import os
img_path = os.path.join(os.getcwd(), 'img.png')

min_score_thresh = 0.35  #@param
max_boxes_to_draw = 200  #@param
line_thickness =   2#@param

import PIL
# Get the largest of height/width and round to 128.
image_size = max(PIL.Image.open(img_path).size)

--2024-12-11 06:05:22--  https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/efficientdet-lite0.tgz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.12.207, 172.217.194.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23972392 (23M) [application/octet-stream]
Saving to: ‘efficientdet-lite0.tgz’

efficientdet-lite0. 100%[===================>]  22.86M  10.7MB/s    in 2.1s    

2024-12-11 06:05:25 (10.7 MB/s) - ‘efficientdet-lite0.tgz’ saved [23972392/23972392]

Use model in /content/automl/efficientdet/efficientdet-lite0
--2024-12-11 06:05:25--  https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png
Resolving user-images.githubusercontent.com (user-images.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to user-images.githubusercontent.

## 0.2 View graph in TensorBoard

In [ ]:
!python -m model_inspect --model_name={MODEL} --logdir=logs &> /dev/null
%load_ext tensorboard
%tensorboard --logdir logs

# 1. inference

## 1.1 Benchmark network latency
There are two types of latency:
network latency and end-to-end latency.

*   network latency: from the first conv op to the network class and box prediction.
*   end-to-end latency: from image preprocessing, network, to the final postprocessing to generate a annotated new image.


In [ ]:
# benchmark network latency
!python -m tf2.inspector --mode=benchmark --model_name={MODEL} --hparams="mixed_precision=true" --only_network

# With colab + Tesla T4 GPU, here are the batch size 1 latency summary:
# D0 (AP=33.5):  14.9ms,  FPS = 67.2   (batch size 8 FPS=)
# D1 (AP=39.6):  22.7ms,  FPS = 44.1   (batch size 8 FPS=)
# D2 (AP=43.0):  27.9ms,  FPS = 35.8   (batch size 8 FPS=)
# D3 (AP=45.8):  48.1ms,  FPS = 20.8   (batch size 8 FPS=)
# D4 (AP=49.4):  81.9ms,  FPS = 12.2   (batch size 8 FPS=)

## 1.2 Benchmark end-to-end latency

In [ ]:
# Benchmark end-to-end latency (: preprocess + network + posprocess).
#
# With colab + Tesla T4 GPU, here are the batch size 1 latency summary:
# D0 (AP=33.5): 22.7ms,  FPS = 43.1   (batch size 4, FPS=)
# D1 (AP=39.6): 34.3ms,  FPS = 29.2   (batch size 4, FPS=)
# D2 (AP=43.0): 42.5ms,  FPS = 23.5   (batch size 4, FPS=)
# D3 (AP=45.8): 64.8ms,  FPS = 15.4   (batch size 4, FPS=)
# D4 (AP=49.4): 93.7ms,  FPS = 10.7   (batch size 4, FPS=)

batch_size = 1  # @param

saved_model_dir = 'savedmodel'
!rm -rf {saved_model_dir}

!python -m tf2.inspector --mode=export --model_name={MODEL} \
  --model_dir={ckpt_path} --saved_model_dir={saved_model_dir} \
  --batch_size={batch_size}  --hparams="mixed_precision=true"
!python -m tf2.inspector --mode=benchmark --model_name={MODEL} \
  --saved_model_dir={saved_model_dir} \
  --batch_size=1  --hparams="mixed_precision=true" --input_image=testdata/img1.jpg


## 1.3 Inference images.

---



In [ ]:
# first export a saved model.
saved_model_dir = 'savedmodel'
!rm -rf {saved_model_dir}
!python -m tf2.inspector --mode=export --model_name={MODEL} \
  --model_dir={ckpt_path} --saved_model_dir={saved_model_dir}

# Then run saved_model_infer to do inference.
# Notably: batch_size, image_size must be the same as when it is exported.
serve_image_out = 'serve_image_out'
!mkdir {serve_image_out}

!python -m tf2.inspector --mode=infer \
  --saved_model_dir={saved_model_dir} \
  --model_name={MODEL}  --input_image=testdata/img1.jpg  \
  --output_image_dir={serve_image_out}

In [ ]:
from IPython import display
display.display(display.Image(os.path.join(serve_image_out, '0.jpg')))

In [ ]:
# In case you need to specify different image size or batch size or #boxes, then
# you need to export a new saved model and run the inferernce.

serve_image_out = 'serve_image_out'
!mkdir {serve_image_out}
saved_model_dir = 'savedmodel'
!rm -rf {saved_model_dir}

# Step 1: export model
!python -m tf2.inspector --mode=export \
  --model_name={MODEL} --model_dir={MODEL} \
  --hparams="image_size=1920x1280" --saved_model_dir={saved_model_dir}

# Step 2: do inference with saved model.
!python -m tf2.inspector --mode=infer \
  --model_name={MODEL} --saved_model_dir={saved_model_dir} \
  --input_image=img.png --output_image_dir={serve_image_out} \

from IPython import display
display.display(display.Image(os.path.join(serve_image_out, '0.jpg')))

## 1.4 Inference video

In [ ]:
# step 0: download video
video_url = 'https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/data/video480p.mov'  # @param
!wget {video_url} -O input.mov

# Step 1: export model
saved_model_dir = 'savedmodel'
!rm -rf {saved_model_dir}

!python -m tf2.inspector --mode=export \
  --model_name={MODEL} --model_dir={MODEL} \
  --saved_model_dir={saved_model_dir} --hparams="mixed_precision=true"

# Step 2: do inference with saved model using saved_model_video
!python -m tf2.inspector --mode=video \
  --model_name={MODEL} \
  --saved_model_dir={saved_model_dir} --hparams="mixed_precision=true" \
  --input_video=input.mov --output_video=output.mov
# Then you can view the output.mov

# 2. TFlite

## 2.1 COCO evaluation on validation set.

In [ ]:
if 'val2017' not in os.listdir():
  !wget http://images.cocodataset.org/zips/val2017.zip
  !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
  !unzip -q val2017.zip
  !unzip annotations_trainval2017.zip

  !mkdir tfrecord
  !python -m dataset.create_coco_tfrecord \
      --image_dir=val2017 \
      --object_annotations_file=annotations/instances_val2017.json \
      --output_file_prefix=tfrecord/val \
      --num_shards=32

## 2.2 TFlite export INT8 model

In [ ]:
# In case you need to specify different image size or batch size or #boxes, then
# you need to export a new saved model and run the inferernce.

serve_image_out = 'serve_image_out'
!mkdir {serve_image_out}
saved_model_dir = 'savedmodel'
!rm -rf {saved_model_dir}

# # Step 1: export model
!python -m tf2.inspector --mode=export --file_pattern=tfrecord/*.tfrecord \
  --model_name={MODEL} --model_dir={MODEL} --num_calibration_steps=100 \
  --saved_model_dir={saved_model_dir} --use_xla --tflite=INT8

# Step 2: do inference with saved model.
!python -m tf2.inspector --mode=infer  --use_xla \
  --model_name={MODEL} --saved_model_dir={saved_model_dir}/int8.tflite \
  --input_image=testdata/img1.jpg --output_image_dir={serve_image_out}

from IPython import display
display.display(display.Image(os.path.join(serve_image_out, '0.jpg')))

In [ ]:
# Evalute on validation set (takes about 10 mins for efficientdet-d0)
!python -m tf2.eval_tflite  \
    --model_name={MODEL}  --tflite_path={saved_model_dir}/int8.tflite   \
    --val_file_pattern=tfrecord/val* \
    --val_json_file=annotations/instances_val2017.json --eval_samples=100

## 2.3 Compile EdgeTPU model (Optional)

In [ ]:
 # install edgetpu compiler
 !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
 !echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
 !sudo apt-get update
 !sudo apt-get install edgetpu-compiler

The EdgeTPU has 8MB of SRAM for caching model paramaters ([more info](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching)). This means that for models that are larger than 8MB, inference time will be increased in order to transfer over model paramaters. One way to avoid this is [Model Pipelining](https://coral.ai/docs/edgetpu/pipeline/) - splitting the model into segments that can have a dedicated EdgeTPU. This can significantly improve latency.

The below table can be used as a reference for the number of Edge TPUs to use - the larger models will not compile for a single TPU as the intermediate tensors can't fit in on-chip memory.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

In [ ]:
NUMBER_OF_TPUS = 1
!edgetpu_compiler {saved_model_dir}/int8.tflite --num_segments=$NUMBER_OF_TPUS

# 3. COCO evaluation

## 3.1 COCO evaluation on validation set.

In [ ]:
if 'val2017' not in os.listdir():
  !wget http://images.cocodataset.org/zips/val2017.zip
  !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
  !unzip -q val2017.zip
  !unzip annotations_trainval2017.zip

  !mkdir tfrecord
  !python -m dataset.create_coco_tfrecord \
      --image_dir=val2017 \
      --object_annotations_file=annotations/instances_val2017.json \
      --output_file_prefix=tfrecord/val \
      --num_shards=32

In [ ]:
# Evalute on validation set (takes about 10 mins for efficientdet-d0)
!python -m tf2.eval  \
    --model_name={MODEL}  --model_dir={ckpt_path}  \
    --val_file_pattern=tfrecord/val*  \
    --val_json_file=annotations/instances_val2017.json

# 4. Training EfficientDets on PASCAL.

## 4.1 Prepare data

In [ ]:
# Get pascal voc 2012 trainval data
import os
if 'VOCdevkit' not in os.listdir():
  !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
  !tar xf VOCtrainval_06-Nov-2007.tar

  !mkdir tfrecord
  !python -m dataset.create_pascal_tfrecord  \
    --data_dir=VOCdevkit --year=VOC2007  --output_path=tfrecord/pascal

# Pascal has 5717 train images with 100 shards epoch, here we use a single shard
# for demo, but users should use all shards pascal-*-of-00100.tfrecord.
file_pattern = 'pascal-00000-of-00100.tfrecord'  # @param
images_per_epoch = 57 * len(tf.io.gfile.glob('tfrecord/' + file_pattern))
images_per_epoch = images_per_epoch // 8 * 8  # round to 64.
print('images_per_epoch = {}'.format(images_per_epoch))

## 4.2 Train Pascal VOC 2007 from ImageNet checkpoint for Backbone.

In [5]:
!mkdir /content/automl/efficientdet/tfrecord
!unzip /content/tfrecord.zip -d /content/automl/efficientdet/tfrecord

Archive:  /content/tfrecord.zip
  inflating: /content/automl/efficientdet/tfrecord/json_train_data.json  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00000-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00001-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00002-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00003-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00004-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00005-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00006-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00007-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00008-of-00010.tfrecord  
  inflating: /content/automl/efficientdet/tfrecord/train_data-00009-of-00010.tfrecord  


In [7]:
!ls tfrecord/train_data-*.tfrecord

tfrecord/train_data-00000-of-00010.tfrecord  tfrecord/train_data-00005-of-00010.tfrecord
tfrecord/train_data-00001-of-00010.tfrecord  tfrecord/train_data-00006-of-00010.tfrecord
tfrecord/train_data-00002-of-00010.tfrecord  tfrecord/train_data-00007-of-00010.tfrecord
tfrecord/train_data-00003-of-00010.tfrecord  tfrecord/train_data-00008-of-00010.tfrecord
tfrecord/train_data-00004-of-00010.tfrecord  tfrecord/train_data-00009-of-00010.tfrecord


In [6]:
import os
import tensorflow as tf

# TFRecord 절대 경로
train_file_pattern = "/content/automl/efficientdet/tfrecord/train_data-*.tfrecord"

# 파일 경로 디버깅
print("Checking train file pattern:", train_file_pattern)
print("Absolute train file pattern:", os.path.abspath(train_file_pattern))

# 파일 존재 여부 확인
train_files = tf.io.gfile.glob(train_file_pattern)
print("Found train files:", train_files)

if not train_files:
    raise ValueError(f"No matching training files for pattern: {train_file_pattern}")

Checking train file pattern: /content/automl/efficientdet/tfrecord/train_data-*.tfrecord
Absolute train file pattern: /content/automl/efficientdet/tfrecord/train_data-*.tfrecord
Found train files: ['/content/automl/efficientdet/tfrecord/train_data-00009-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00004-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00002-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00008-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00001-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00005-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00003-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00007-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00000-of-00010.tfrecord', '/content/automl/efficientdet/tfrecord/train_data-00006-of-00010.tfrecord']


In [16]:
# Train efficientdet from scratch with backbone checkpoint.
backbone_name = {
    'efficientdet-d0': 'efficientnet-b0',
    'efficientdet-d1': 'efficientnet-b1',
    'efficientdet-d2': 'efficientnet-b2',
    'efficientdet-d3': 'efficientnet-b3',
    'efficientdet-d4': 'efficientnet-b4',
    'efficientdet-d5': 'efficientnet-b5',
    'efficientdet-d6': 'efficientnet-b6',
    'efficientdet-d7': 'efficientnet-b6',
    'efficientdet-lite0': 'efficientnet-lite0',
    'efficientdet-lite1': 'efficientnet-lite1',
    'efficientdet-lite2': 'efficientnet-lite2',
    'efficientdet-lite3': 'efficientnet-lite3',
    'efficientdet-lite3x': 'efficientnet-lite3',
    'efficientdet-lite4': 'efficientnet-lite4',
}[MODEL]


# generating train tfrecord is large, so we skip the execution here.
import os
if backbone_name not in os.listdir():
  if backbone_name.find('lite'):
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/lite/{backbone_name}.tar.gz
  else:
    !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/ckptsaug/{backbone_name}.tar.gz
  !tar xf {backbone_name}.tar.gz

!mkdir /tmp/model_dir

!python -m tf2.train --mode=train \
    --train_file_pattern="/content/automl/efficientdet/tfrecord/train_data-*.tfrecord" \
    --model_name="efficientdet-lite0" \
    --model_dir="/content/automl/efficientdet-lite0-model" \
    --pretrained_ckpt="/content/automl/efficientdet/efficientdet-lite0" \
    --batch_size=8 \
    --num_examples_per_epoch=16 \
    --num_epochs=100 \
    --hparams="num_classes=1,moving_average_decay=0,mixed_precision=true"

mkdir: cannot create directory ‘/tmp/model_dir’: File exists
2024-12-11 06:39:02.126597: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:39:02.126660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:39:02.128366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:39:03.459083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-12-11 06:39:07.010678: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable i

In [9]:
!pwd

/content/automl/efficientdet


In [19]:
!python inspector.py \
    --mode=export \
    --model_name=efficientdet-lite0 \
    --model_dir=/content/automl/efficientdet-lite0-model \
    --saved_model_dir=/content/automl/efficientdet-lite0-savedmodel \
    --tflite=INT8 \
    --file_pattern="/content/automl/efficientdet/tfrecord/train_data-*.tfrecord" \
    --num_calibration_steps=500 \
    --hparams="num_classes=1"

2024-12-11 06:53:23.197490: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:53:23.197545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:53:23.199069: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:53:24.314074: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I1211 06:53:26.614176 134306054221824 efficientnet_lite_builder.py:100] global_params= GlobalParams(batch_norm_momentum=0.99, batch_norm_epsilon=0.001, dropout_rate=0.2, data_format='channels_last', num_classes=1000, width_coefficient=1.0, d

In [21]:
!python inspector.py \
    --mode=export \
    --model_name=efficientdet-lite0 \
    --model_dir=/content/automl/efficientdet-lite0-model \
    --saved_model_dir=/content/automl/efficientdet-lite0-savedmodel \
    --tflite=FP16 \
    --hparams="num_classes=1"

2024-12-11 06:56:19.332239: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 06:56:19.332352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 06:56:19.334382: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 06:56:20.974288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I1211 06:56:24.072198 135278153166848 efficientnet_lite_builder.py:100] global_params= GlobalParams(batch_norm_momentum=0.99, batch_norm_epsilon=0.001, dropout_rate=0.2, data_format='channels_last', num_classes=1000, width_coefficient=1.0, d

In [20]:
!ls -lh /content/automl/efficientdet-lite0-savedmodel/int8.tflite

-rw-r--r-- 1 root root 4.2M Dec 11 06:54 /content/automl/efficientdet-lite0-savedmodel/int8.tflite


## 4.3 Train Pascal VOC 2007 from COCO checkpoint for the whole net.

In [ ]:
# generating train tfrecord is large, so we skip the execution here.
import os
if MODEL not in os.listdir():
  download(MODEL)

!mkdir /tmp/model_dir/
# key option: use --ckpt rather than --backbone_ckpt.
!python -m tf2.train --mode=traineval \
    --train_file_pattern=tfrecord/{file_pattern} \
    --val_file_pattern=tfrecord/{file_pattern} \
    --model_name={MODEL} \
    --model_dir=/tmp/model_dir/{MODEL}-finetune \
    --pretrained_ckpt={MODEL} \
    --batch_size=16 \
    --eval_samples={images_per_epoch}  \
    --num_examples_per_epoch={images_per_epoch}  --num_epochs=1  \
    --hparams="num_classes=20,moving_average_decay=0,mixed_precision=true"

## 4.4 View tensorboard for loss and accuracy.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/model_dir/
# Notably, this is just a demo with almost zero accuracy due to very limited
# training steps, but we can see finetuning has smaller loss than training
# from scratch at the begining.

## 5. Export to onnx


In [ ]:
!pip install tf2onnx

In [ ]:
!python -m tf2.inspector --mode=export --model_name={MODEL} --model_dir={MODEL} --saved_model_dir={saved_model_dir} --hparams="nms_configs.method='hard', nms_configs.iou_thresh=0.5, nms_configs.sigma=0.0"

In [ ]:
!python -m tf2onnx.convert --saved-model={saved_model_dir} --output={saved_model_dir}/model.onnx  --opset=11